# Simulations of yields for various product and substrate combinations

This is the supplementary file for the article Becker et al. (2022) *[Link follows]* for Table 1. We tested the yields of production for various product and substrate yields in the genome scale metabolic model (GSMM) of *U. maydis* iUma22 [[Liebal et al., 2022](https://doi.org/10.3390/jof8050524)].

In [1]:
import pandas as pd
import numpy as np
import re
import os
from datetime import date

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# loading cobrapy, a library dedicated to the analysis of genome scale metabolic models
from cobra.io import read_sbml_model
from cobra import Reaction

Today = date.today().strftime('%y%m%d')

In [2]:
def CoSub(Flux, C1, C2, R):
    '''
    Defining the ratios of cosubstrates.
    
    Arguments:
    Flux: float, total C-mole flux
    C1: int, carbon atoms in substrate 1
    C2: int, carbon atoms in substrate 2
    R: float, flux ratio of both substrate: R = C1*F1 / (C2*F2)
    
    Return:
    F1, F2: float, molar concentrations of substrate 1 and substrate 2.
    '''
    F1 = Flux / (C1*(1+1/R))
    F2 = F1*C1 / (R*C2)
    return F1, F2

def countCinFormula(formula):
    '''
    Counting C-atoms of metabolite in cobra model
    
    Argument:
    formula: string, the metabolite formula from the cobra model
    
    Return:
    int, number of C-atoms in formula
    '''
    return int(re.search(r'C(.*)H',formula).group(1)) if re.search(r'C(.*)H',formula).group(1) else int(1)

def set_ObjiUma(model, Product):
    '''
    Adding boundary flux for a desired product and setting the objective reaction to the product in the model.
    
    Arguments:
    model: cobra model, iUma22
    Product: string, model metabolite ID
    
    Return:
    cobra model, updated with a demand flow for the product and optimization of the new demand reaction.
    '''
    model.add_boundary(model.metabolites.get_by_id(Product), type='demand')
    model.objective = 'DM_{}'.format(Product)
    return model

def set_SubiUma(model, Sub, Flux, Ratio):
    '''
    Setting the desired substrate in the model.
    
    Arguments:
    model: cobra model, iUma22
    Sub: string/list, model metabolite ID
    Flux: float, desired substrate uptake rate
    Ratio: float, when Sub is a list, then uptake rates are balanced according to the Ratio.
    
    Return:
    cobra model, updated with designed (co-) substrate uptake rates.
    '''
    Catoms = [countCinFormula(model.metabolites.get_by_id(subi).formula) for subi in Sub]
    Cup = Flux/np.sum(Catoms)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
    if len(Sub)<2:
        model.reactions.get_by_id('EX_{}'.format(Sub[0])).lower_bound = -Cup
    else:
        C1 = Catoms[0]
        C2 = Catoms[1]
        F1,F2 = CoSub(Flux, C1, C2, Ratio)
        for subi, fluxi in zip(Sub, [F1,F2]):
            model.reactions.get_by_id('EX_{}'.format(subi)).lower_bound = -fluxi
    return model

In [9]:
# generating cobra variable from SBML/xml file
# assuming the base path is in 'model' of a cloned git-directory of iUma22.
iUma_File = os.path.join('..','model','iUma22v1.1.xml')
model = read_sbml_model(iUma_File)

## Adding missing reactions
We added and removed reactions:
1. Enabling substrate uptake of 2-oxo-butanate and formate, no reference as the cytoplasmic metabolites exist in the model
2. A pseudo reaction enables the conversion from mannose-6P to mannitol, reference: by Schipper, 2009: https://d-nb.info/1000245756/34
3. Enabling cytoplasmic ICL activity, reference: https://www.genome.jp/dbget-bin/www_bget?uma:UMAG_01892+uma:UMAG_04285
4. The best yield of itaconate from glucose is 5/6 of C-moles. An unrealistic cycle via oxaloacetate, threonine, acetaldehyde raises the yield. This pathway is broken by disabling the acylcarnitine-translocase between mitochondria and cytoplasm.
5. Succinate is more reduced compared to malate and should thus be produced in a lower yield. To enable the lower yield for succinate, the methanol dehydrogenase (RXN-11453) and the pyridoxalkinase (PDXl) are deactivated.

In [8]:
# Substrate uptake for 2oxo-butanate
# allowing co2 uptake
model.reactions.get_by_id('EX_co2_e').lower_bound = -10
# Substrate uptake for 2-oxo-butanate
model.add_boundary(model.metabolites.get_by_id('2obut_e'), type="exchange", lb=0)
# Substrate uptake for formate
model.add_boundary(model.metabolites.get_by_id('for_e'), type="exchange", lb=0)


# Mannitol production
# https://biocyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWY-3881
# Reported Mannitol pathway genes in U. maydis by Schipper, 2009:
# https://d-nb.info/1000245756/34
ID = 'PseudoMan6P2Mnl'
if ID not in model.reactions:
    reaction = Reaction(ID)
    reaction.name = 'Pseudoreaction from Mannose-6-P to Mannitol'
    reaction.lower_bound = 0.
    reaction.upper_bound = 1000.
#     reaction.annotation = {'biocyc':'MANNPISOM-RXN', 'brenda': '5.3.1.8'}
    model.add_reactions([reaction])
    model.reactions.get_by_id(ID).build_reaction_from_string('MANNOSE-6P_c + nadph_c + h_c + h2o_c --> mnl_c + nadp_c + pi_c')

    print(model.reactions.get_by_id(ID).id)
    print(model.reactions.get_by_id(ID).reaction)
    print(model.reactions.get_by_id(ID).check_mass_balance())
    

# Isocitrate lyase
# https://www.genome.jp/dbget-bin/www_bget?uma:UMAG_01892+uma:UMAG_04285
ID = 'ICL'
if ID not in model.reactions:
    reaction = Reaction(ID)
    reaction.name = 'Isocitrate lyase'
    reaction.lower_bound = 0.
    reaction.upper_bound = 1000.
#     reaction.annotation = {'biocyc':'MANNPISOM-RXN', 'brenda': '5.3.1.8'}
    model.add_reactions([reaction])
    model.reactions.get_by_id(ID).build_reaction_from_string('icit_c --> succ_c + glx_c')

    print(model.reactions.get_by_id(ID).id)
    print(model.reactions.get_by_id(ID).reaction)
    print(model.reactions.get_by_id(ID).check_mass_balance())

    

# deactivating acylcarniting translocase
model.reactions.get_by_id('TRANS_O-ACETYLCARNITINE-Translocase').bounds = 0,0

# formate, meoh cycle for high malate production
model.reactions.get_by_id('RXN-11453').bounds = 0,0
model.reactions.get_by_id('PRDXl').bounds = 0,0

ValueError: Boundary reaction 'EX_2obut_e' already exists.

### Checking the details for metabolite names

In [12]:
# Searching for metabolite IDs
CheckMet = 'formate'
[print('{}:\t{}:\t{}:\t{}\n'.format(MetID.name, MetID.id, MetID.formula, MetID.annotation)) for MetID in model.metabolites.query(CheckMet, 'name')];

formate:	for_e:	CHO2:	{'sbo': 'SBO:0000247', 'biocyc': 'META:FORMATE', 'cas': '64-18-6', 'chebi': '15740', 'chemspider': '277', 'hmdb': 'HMDB00142', 'inchi': 'InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3)/p-1', 'kegg.compound': 'C00058', 'pubchem.compound': '283'}

formate:	for_c:	CHO2:	{'sbo': 'SBO:0000247', 'biocyc': 'META:FORMATE', 'cas': '64-18-6', 'chebi': '15740', 'chemspider': '277', 'hmdb': 'HMDB00142', 'inchi': 'InChI=1S/CH2O2/c2-1-3/h1H,(H,2,3)/p-1', 'kegg.compound': 'C00058', 'pubchem.compound': '283'}



### List of substrates and products being tested

In [13]:
ProList = ['itacon_c','succ_c','mal__L_c', 'eryth_c', 'mnl_c','hex2manerythol_c','UA_A_c', 'CPD-15924_c'] # hexACP_c -> UA_A_c, CPD-13014_c: tributyrin
SubList = [['ac_e'], ['for_e'], ['2obut_e'], ['glc__D_e'], ['glyc_e'], ['xyl__D_e'], ['glc__D_e', 'for_e'], ['glc__D_e', 'ac_e']] #['ac_e'], ['for_e'], 
SubProCom = [(x,y) for x in SubList for y in ProList]

In [14]:
Solution_df = pd.DataFrame(columns=['Substrate', 'Product', 'Yield'])
for SubProi in SubProCom:
    Substrate_lst = SubProi[0]
    Product_str = SubProi[1]
    CnumPro = countCinFormula(model.metabolites.get_by_id(Product_str).formula)
    F = 100 # C-mole/gDW/h
    R = 5
    with model as md:
        md = set_SubiUma(md, Substrate_lst, F, R)
        md = set_ObjiUma(md, Product_str)
        sol = md.optimize()
        Yield = sol.fluxes['DM_{}'.format(Product_str)]*CnumPro/F
        Solution_df = Solution_df.append({'Substrate': str(Substrate_lst), 'Product': Product_str, 'Yield': round(Yield,2)}, ignore_index=True)
        mybnd = [md.reactions.get_by_id('EX_{}'.format(subi)).lower_bound for subi in Substrate_lst]
        print('{} production on {}: {:.2f}'.format(Product_str, Substrate_lst, Yield))

itacon_c production on ['ac_e']: 0.00
succ_c production on ['ac_e']: 0.00
mal__L_c production on ['ac_e']: 0.00
eryth_c production on ['ac_e']: 0.00
mnl_c production on ['ac_e']: 0.00
hex2manerythol_c production on ['ac_e']: 0.00
UA_A_c production on ['ac_e']: 0.00
CPD-15924_c production on ['ac_e']: 0.00
itacon_c production on ['for_e']: 0.00
succ_c production on ['for_e']: 0.00
mal__L_c production on ['for_e']: 0.00
eryth_c production on ['for_e']: 0.00
mnl_c production on ['for_e']: 0.00
hex2manerythol_c production on ['for_e']: 0.00
UA_A_c production on ['for_e']: 0.00
CPD-15924_c production on ['for_e']: 0.00
itacon_c production on ['2obut_e']: 0.83
succ_c production on ['2obut_e']: 0.95
mal__L_c production on ['2obut_e']: 1.00
eryth_c production on ['2obut_e']: 0.32
mnl_c production on ['2obut_e']: 0.37
hex2manerythol_c production on ['2obut_e']: 0.39
UA_A_c production on ['2obut_e']: 0.43
CPD-15924_c production on ['2obut_e']: 0.40
itacon_c production on ['glc__D_e']: 0.95
succ_

In [15]:
Solution_Tab = pd.pivot(Solution_df, values='Yield', index='Substrate', columns='Product')
Solution_Tab['Mean-Substrates'] = round(Solution_Tab.mean(axis=1), 2)
Mean = Solution_Tab.mean()
Mean.name='Mean-Products'
Solution_Tab = round(Solution_Tab.append(Mean), 2)
Solution_Tab

Product,CPD-15924_c,UA_A_c,eryth_c,hex2manerythol_c,itacon_c,mal__L_c,mnl_c,succ_c,Mean-Substrates
Substrate,,,,,,,,,
['2obut_e'],0.40,0.43,0.32,0.39,0.83,1.00,0.37,0.95,0.59
['ac_e'],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
['for_e'],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"['glc__D_e', 'ac_e']",0.62,0.69,0.73,0.76,0.97,1.10,0.80,0.99,0.83
"['glc__D_e', 'for_e']",0.55,0.61,0.73,0.69,0.89,1.05,0.72,0.92,0.77
['glc__D_e'],0.67,0.73,0.88,0.82,0.95,1.10,0.86,0.97,0.87
['glyc_e'],0.75,0.81,0.88,0.91,1.05,1.10,0.96,1.10,0.94
['xyl__D_e'],0.69,0.75,0.79,0.84,1.00,1.10,0.87,1.01,0.88
Mean-Products,0.46,0.50,0.54,0.55,0.71,0.81,0.57,0.74,0.61


In [101]:
Solution_Tab.to_excel('{}_SubProdTable.xlsx'.format(Today))

In [86]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -100/6
    #     model.reactions.get_by_id('EX_for_e').lower_bound = -0
    model = set_ObjiUma(model, 'succ_c')
    #     model.objective = 'EX_succ_e'
    mySol = model.optimize()
    print(model.summary())

Objective
1.0 DM_succ_c = 27.499999999999996

Uptake
------
Metabolite    Reaction  Flux  C-Number C-Flux
     co2_e    EX_co2_e    10         1  9.09%
  glc__D_e EX_glc__D_e 16.67         6 90.91%
      o2_e     EX_o2_e  3.75         0  0.00%

Secretion
---------
Metabolite  Reaction  Flux  C-Number  C-Flux
    succ_c DM_succ_c -27.5         4 100.00%
     h2o_e  EX_h2o_e -17.5         0   0.00%



In [ ]:
with model as model:
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = -100/6
    model.reactions.get_by_id('EX_for_e').lower_bound = -0
    model = set_ObjiUma(model, 'succ_c')
#     model.objective = 'EX_succ_e'
    print(model.summary())

In [1]:
ratio = np.linspace(8,2,2)
mySol = list()
for iratio in ratio:
    glc_up, ace_up = CoSub(100,6,2,iratio)
    print('glc:{}, ace:{}'.format(glc_up, ace_up))
    with model as model:
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = -glc_up
        model.reactions.get_by_id('EX_ac_e').lower_bound = -ace_up
        # model = set_ObjiUma(model, 'succ_c')
    #     model.objective = 'EX_succ_e'
        print(model.summary())
        mySol.append(model.optimize().fluxes)
    print(glc_up, ace_up)
    

NameError: name 'np' is not defined

In [14]:
list(model.metabolites.get_by_id('co2_e').reactions)

[<Reaction CO2tex at 0x7fd310894be0>, <Reaction EX_co2_e at 0x7fd310894c70>]

In [87]:
[print('{}\t{}\t{}'.format(rct.id, rct.reaction, mySol.fluxes[rct.id])) for rct in model.metabolites.get_by_id('meoh_c').reactions if np.abs(mySol.fluxes[rct.id])>1];

PRDXl	h2o2_c + meoh_c <=> fald_c + 2.0 h2o_c	-107.3148148148148
ALCD1	meoh_c + nad_c --> fald_c + h_c + nadh_c	107.3148148148148


In [68]:
model.metabolites.query('aceta','name')

[<Metabolite id3acald_c at 0x2249e351bb0>,
 <Metabolite pac_c at 0x2249e3c7970>,
 <Metabolite fpram_c at 0x2249e3d18b0>,
 <Metabolite CPD-12708_c at 0x2249e3ec9d0>,
 <Metabolite 4mlacac_c at 0x2249e3ece80>,
 <Metabolite 4fumacac_c at 0x2249e3eceb0>,
 <Metabolite acald_c at 0x2249e418df0>,
 <Metabolite oaa_c at 0x2249e418fa0>,
 <Metabolite oaa_m at 0x2249e418d00>,
 <Metabolite ind3ac_c at 0x2249e425ca0>,
 <Metabolite ad_c at 0x2249e425ee0>,
 <Metabolite acac_c at 0x2249e4416d0>,
 <Metabolite ac_e at 0x2249e4419d0>,
 <Metabolite ac_c at 0x2249e441a00>,
 <Metabolite CPD-12709_e at 0x2249e441b80>,
 <Metabolite CPD-12709_c at 0x2249e441bb0>]

In [95]:
model.reactions.get_by_id('ICL')

KeyError: 'ICL'